# Overview
- 特徴量抽出するライブラリのイメージを作成

# Import everything I need

In [1]:
import sys
import librosa 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import inspect
import importlib
import glob

sys.path.insert(0, 'lib')
from feat_librosa import FeatLibrosa
from feat_mfcc import FeatMFCC

/usr/local/lib/python3.6/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


# My function

In [2]:
# ズバリの記事があった(from ugajin)
#https://qiita.com/m_mizutani/items/4c37749e93491a37d977
def load_classes(dname):
    '''
    ref: https://qiita.com/m_mizutani/items/4c37749e93491a37d977
    '''
    class_list = []
    # *.pyを読み込む
    for fpath in glob.glob(os.path.join(dname, '*.py')):
        # 拡張子を除去＆ファイルパスのセパレータ (Unix系なら'/') を '.' に置き換え
        cpath = os.path.splitext(fpath)[0].replace(os.path.sep, '.')
        # モジュールとして読み込み
        mod = importlib.import_module(cpath)
        # 読み込んだモジュールに含まれるクラスを呼び出す
        for cls_name, cls in inspect.getmembers(mod, inspect.isclass):
            # (ディレクトリパス含むクラス名, クラス本体) という形式で格納
            class_list.append(('{0}.{1}'.format(cpath, cls_name), cls))

    return class_list


# Preparation

In [3]:
filename = librosa.util.example_audio_file()
data, sr = librosa.load(filename, sr=None)

# Trial

In [4]:
class HmcommFeatureExtraction:
    def __init__(self):
        self.sampling_rate = 44100
        self.window_size_sec = 0.5
        self.window_stride_sec = 0.25
    
    def feature_extraction(self, signal, feat_sets, **kwargs):
        # lib内のクラスを取得
        clist = load_classes('lib')
        clist.sort()
        
        df_list = []
        for i in range(len(clist)):
            # Feat*** classを取得
            class_ = clist[i][1](self.sampling_rate, self.window_size_sec, self.window_stride_sec)
            name = class_.name
            default_option = class_.default_option
            option = kwargs.get(name, default_option)
            df_feats = class_.feature_extraction(signal, **option)
            df_list.append(df_feats)
        
        return pd.concat(df_list, axis=1)

In [5]:
fe = HmcommFeatureExtraction()
fe.sampling_rate = 44100
fe.window_size_sec = 0.5

# df_feat = fe.feature_extraction(data, feat_sets='all')
df_feat = fe.feature_extraction(data, feat_sets='all')
df_feat.head()

,librosa_centroid,librosa_rolloff,mfcc_00,mfcc_01,mfcc_02,mfcc_03,mfcc_04,mfcc_05,mfcc_06,mfcc_07,...,mfcc_10,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19
0,1586.251720,1980.0,-159.216156,163.961807,37.366264,14.662003,9.271687,10.448686,16.696106,12.573850,...,10.147247,8.404415,5.111315,4.635045,5.034909,6.858226,7.556915,5.880576,1.484067,-3.523701
1,768.158733,1136.0,-179.706039,179.745972,66.054710,-4.973018,-6.118852,14.803446,18.070827,10.055754,...,9.216499,8.249978,9.149309,8.934200,6.486597,4.946148,5.291379,5.089251,3.464425,1.943611
2,440.427527,694.0,-139.558960,221.975021,81.687454,-2.429063,-2.767245,17.909702,16.269829,4.282356,...,-7.942372,-5.142995,-0.680546,-2.759790,-6.834208,-3.396309,4.591705,7.593139,5.233050,3.748049
3,546.184605,896.0,-170.453491,192.227051,75.869247,0.284083,-4.622998,14.205166,15.033255,4.449258,...,3.032630,-1.482779,-1.133784,1.255097,0.122856,-1.583692,1.157060,5.580401,6.151880,3.812955
4,563.347235,1010.0,-162.858124,198.169144,71.786812,-9.577950,-10.098435,19.332977,26.851265,13.871593,...,6.362561,3.524467,2.080387,2.578875,2.292722,1.839322,3.124622,4.686130,4.077188,1.817299


In [25]:
params = {
    'Librosa': {'window': 'hamming'},
    'MFCC': {'n_mfcc':5},
}

In [26]:
fe = HmcommFeatureExtraction()
fe.sampling_rate = 44100
fe.window_size_sec = 0.5

# df_feat = fe.feature_extraction(data, feat_sets='all')
df_feat = fe.feature_extraction(data, feat_sets='all', **params)

In [27]:
df_feat.head()

,librosa_centroid,mfcc_00,mfcc_01,mfcc_02,mfcc_03,mfcc_04
0,1555.634697,-159.216156,163.961807,37.366264,14.662003,9.271687
1,819.211418,-179.706039,179.745972,66.054710,-4.973018,-6.118852
2,456.371161,-139.558960,221.975021,81.687454,-2.429063,-2.767245
3,581.062371,-170.453491,192.227051,75.869247,0.284083,-4.622998
4,579.534943,-162.858124,198.169144,71.786812,-9.577950,-10.098435


In [9]:
params = {
    'Librosa': {'window': 'hamming'},
    'MFCC': {'n_mfcc':7, 'dct_type':1},
}

In [10]:
fe = HmcommFeatureExtraction()
fe.sampling_rate = 44100

df_feat = fe.feature_extraction(data, feat_sets='all', **params)

---

In [7]:
fe = HmcommFeatureExtraction()
fe.sampling_rate = 44100

# df_feat = fe.feature_extraction(data, feat_sets='all')
df_feat = fe.feature_extraction(data, feat_sets=['MFCC', 'open_smile'])
df_feat.head()

,librosa_centroid,mfcc_00,mfcc_01,mfcc_02,mfcc_03,mfcc_04,mfcc_05,mfcc_06,mfcc_07,mfcc_08,...,mfcc_10,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19
0,1586.251720,-159.216156,163.961807,37.366264,14.662003,9.271687,10.448686,16.696106,12.573850,10.602503,...,10.147247,8.404415,5.111315,4.635045,5.034909,6.858226,7.556915,5.880576,1.484067,-3.523701
1,768.158733,-179.706039,179.745972,66.054710,-4.973018,-6.118852,14.803446,18.070827,10.055754,7.743094,...,9.216499,8.249978,9.149309,8.934200,6.486597,4.946148,5.291379,5.089251,3.464425,1.943611
2,440.427527,-139.558960,221.975021,81.687454,-2.429063,-2.767245,17.909702,16.269829,4.282356,-1.542549,...,-7.942372,-5.142995,-0.680546,-2.759790,-6.834208,-3.396309,4.591705,7.593139,5.233050,3.748049
3,546.184605,-170.453491,192.227051,75.869247,0.284083,-4.622998,14.205166,15.033255,4.449258,2.018257,...,3.032630,-1.482779,-1.133784,1.255097,0.122856,-1.583692,1.157060,5.580401,6.151880,3.812955
4,563.347235,-162.858124,198.169144,71.786812,-9.577950,-10.098435,19.332977,26.851265,13.871593,5.116270,...,6.362561,3.524467,2.080387,2.578875,2.292722,1.839322,3.124622,4.686130,4.077188,1.817299


---

In [9]:
params = {
    'Librosa': {'window': 'hamming'},
    'MFCC': {'n_mfcc':7, 'dct_type':1},
}

In [10]:
fe = HmcommFeatureExtraction()
fe.sampling_rate = 44100

df_feat = fe.feature_extraction(data, feat_sets='all', **params)

In [11]:
df_feat.head()

,librosa_centroid,mfcc_00,mfcc_01,mfcc_02,mfcc_03,mfcc_04,mfcc_05,mfcc_06
0,1555.634697,-159.446091,163.507462,36.346043,15.791849,8.595157,11.531868,16.170794
1,819.211418,-179.965714,179.523865,65.182152,-3.677998,-6.480180,15.799837,17.513269
2,456.371161,-139.727264,221.990921,81.302368,-0.413428,-2.268368,19.882109,16.857586
3,581.062371,-170.684967,192.085770,75.177345,1.880118,-4.566643,15.702587,14.997705
4,579.534943,-163.163025,197.940826,71.000847,-8.185971,-10.412234,20.395559,26.505968


In [11]:
fe = HmcommFeatureExtraction()
fe.sampling_rate = 44100
fe.window_size_sec = 0.5

df_feat = fe.feature_extraction(data, feat_sets='all')

In [6]:
df_feat

,librosa_centroid,librosa_rolloff,librosa_mfcc_00,librosa_mfcc_01,librosa_mfcc_02,librosa_mfcc_03,librosa_mfcc_04,librosa_mfcc_05,librosa_mfcc_06,librosa_mfcc_07,...,librosa_mfcc_10,librosa_mfcc_11,librosa_mfcc_12,librosa_mfcc_13,librosa_mfcc_14,librosa_mfcc_15,librosa_mfcc_16,librosa_mfcc_17,librosa_mfcc_18,librosa_mfcc_19
0,1586.251720,1980.0,-159.216156,163.961807,37.366264,14.662003,9.271687,10.448686,16.696106,12.573850,...,10.147247,8.404415,5.111315,4.635045,5.034909,6.858226,7.556915,5.880576,1.484067,-3.523701
1,768.158733,1136.0,-179.706039,179.745972,66.054710,-4.973018,-6.118852,14.803446,18.070827,10.055754,...,9.216499,8.249978,9.149309,8.934200,6.486597,4.946148,5.291379,5.089251,3.464425,1.943611
2,440.427527,694.0,-139.558960,221.975021,81.687454,-2.429063,-2.767245,17.909702,16.269829,4.282356,...,-7.942372,-5.142995,-0.680546,-2.759790,-6.834208,-3.396309,4.591705,7.593139,5.233050,3.748049
3,546.184605,896.0,-170.453491,192.227051,75.869247,0.284083,-4.622998,14.205166,15.033255,4.449258,...,3.032630,-1.482779,-1.133784,1.255097,0.122856,-1.583692,1.157060,5.580401,6.151880,3.812955
4,563.347235,1010.0,-162.858124,198.169144,71.786812,-9.577950,-10.098435,19.332977,26.851265,13.871593,...,6.362561,3.524467,2.080387,2.578875,2.292722,1.839322,3.124622,4.686130,4.077188,1.817299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,535.642609,662.0,-319.378571,37.709755,33.324879,27.041693,20.132517,13.917799,9.448887,7.263423,...,10.848011,12.224936,12.019887,9.762704,5.565328,0.092559,-5.610854,-10.390567,-13.279591,-13.745184
242,1032.785423,664.0,-332.755280,19.519876,17.185467,13.908375,10.441946,7.551741,5.819152,5.493865,...,9.921049,11.024637,10.890112,9.272460,6.315177,2.516204,-1.398520,-4.660660,-6.655802,-7.079402
243,2263.967912,6234.0,-342.491516,6.458313,6.077760,5.537746,4.954200,4.445308,4.101463,3.962473,...,4.320227,4.364418,4.207015,3.807645,3.185699,2.416042,1.610460,0.889652,0.353048,0.054177
244,2645.247194,6694.0,-346.143463,1.428724,1.422625,1.412481,1.398329,1.380213,1.358195,1.332346,...,1.232746,1.192559,1.149093,1.102489,1.052904,1.000498,0.945444,0.887928,0.828134,0.766259


---

In [12]:
params = {
    'Librosa': {'window': 'hamming'},
    'MFCC': {'n_mfcc':7, 'dct_type':1},
}

In [9]:
fe = HmcommFeatureExtraction()
fe.sampling_rate = sr

df_feat = fe.feature_extraction(data, feat_sets='all', **params)

In [10]:
df_feat

,librosa_centroid,librosa_rolloff,librosa_mfcc_00,librosa_mfcc_01,librosa_mfcc_02,librosa_mfcc_03,librosa_mfcc_04,librosa_mfcc_05,librosa_mfcc_06
0,1555.634697,1980.0,-159.216156,163.961807,37.366264,14.662003,9.271687,10.448686,16.696106
1,819.211418,1136.0,-179.706039,179.745972,66.054710,-4.973018,-6.118852,14.803446,18.070827
2,456.371161,694.0,-139.558960,221.975021,81.687454,-2.429063,-2.767245,17.909702,16.269829
3,581.062371,896.0,-170.453491,192.227051,75.869247,0.284083,-4.622998,14.205166,15.033255
4,579.534943,1010.0,-162.858124,198.169144,71.786812,-9.577950,-10.098435,19.332977,26.851265
...,...,...,...,...,...,...,...,...,...
241,610.970134,662.0,-319.378571,37.709755,33.324879,27.041693,20.132517,13.917799,9.448887
242,1113.243387,664.0,-332.755280,19.519876,17.185467,13.908375,10.441946,7.551741,5.819152
243,2205.583879,6234.0,-342.491516,6.458313,6.077760,5.537746,4.954200,4.445308,4.101463
244,2737.237004,6694.0,-346.143463,1.428724,1.422625,1.412481,1.398329,1.380213,1.358195
